# Imports

In [1]:
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from scipy import stats
from scipy.sparse import csr_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error

import xgb
from catboost import CatBoostRegressor
from xgboost import XGBRegressor, DMatrix
from lightgbm import LGBMRegressor, Dataset

import joblib
import optuna
from optuna.samplers import TPESampler


plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
%reload_ext autotime

time: 0 ns (started: 2024-06-27 13:47:31 +09:00)


# Define Functions

In [2]:
def show_all_columns() -> None:
    """Set pandas display options to show all columns."""
    pd.set_option('display.max_columns', None)  # Show all columns
    pd.set_option('display.expand_frame_repr', False)  # Prevent column wrapping
    pd.set_option('display.max_colwidth', None)  # Show full column content
    
def set_random_seed(seed_value: int) -> None:
    np.random.seed(seed_value)
    
def add_date_features(df: pd.DataFrame, date_column: str, prefix: str) -> pd.DataFrame:
    # Ensure the date column is in datetime format
    if not pd.api.types.is_datetime64_any_dtype(df[date_column]):
        df[date_column] = pd.to_datetime(df[date_column])
        
    # Adding date features
    df[f'{prefix}.year'] = df[date_column].dt.year
    df[f'{prefix}.month'] = df[date_column].dt.month
    df[f'{prefix}.day'] = df[date_column].dt.day
    df[f'{prefix}.quarter'] = df[date_column].dt.quarter
    df[f'{prefix}.week'] = df[date_column].dt.isocalendar().week.astype(np.int32)
    
    return df

def add_time_features(df: pd.DataFrame, hour_column: str, prefix: str = 'hour') -> pd.DataFrame:
    df[f'{prefix}_sin_time'] = np.sin(2 * np.pi * df[hour_column] / 24)
    df[f'{prefix}_cos_time'] = np.cos(2 * np.pi * df[hour_column] / 24)
    return df
    
def calculate_cdd(df: pd.DataFrame, temp_column: str, base_temp: float = 24.0) -> pd.DataFrame:
    df['CDD'] = df[temp_column].apply(lambda x: max(0, x - base_temp))
    return df

def calculate_thi(df: pd.DataFrame, temp_colum: str, humidity_column: str) -> pd.DataFrame:
    df['THI'] = 1.8 * df[temp_colum] - 0.55 * (1 - (df[humidity_column] / 100)) * (1.8 * df[temp_colum] - 26) + 32
    return df

def clean_column_name_train(df: pd.DataFrame, prefix_to_remove: str='electric_train.') -> pd.DataFrame:
    """
    데이터프레임의 칼럼명에서 접두사를 제거하고, 칼럼명을 매핑에 따라 변경.

    Args:
        df (pd.DataFrame): 원본 데이터프레임

    Returns:
        pd.DataFrame: 칼럼명이 정리된 데이터프레임
    """

    mapping = {
    'tm': 'TM', 'hh24': 'HH24', 'weekday': 'weekday', 'week_name': 'week_name', 'sum_qctr': 'sum_qctr',
    'n': 'n', 'sum_load': 'sum_load', 'n_mean_load': 'n_mean_load', 'elec': 'elec',
    'num': 'NUM', 'stn': 'STN', 'nph_ta': 'nph_ta', 'nph_hm': 'nph_hm', 'nph_ws_10m': 'nph_ws_10m',
    'nph_rn_60m': 'nph_rn_60m', 'nph_ta_chi': 'nph_ta_chi'
    }

    # 접두사 제거
    df.columns = [col.replace(prefix_to_remove, '') for col in df.columns]

    # 칼럼명 변경
    df = df.rename(columns=mapping)

    return df

def get_rscore(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """
    실제 값과 예측 값 사이의 피어슨 상관 계수(R-score)를 계산.
    
    이 함수는 두 데이터 집합 간의 선형 상관 관계를 측정하는 피어슨 상관 계수를 계산한다.
    반환되는 값은 -1과 1 사이이며:
     - 1은 완벽한 양의 선형 관계
     - -1은 완벽한 음의 선형 관계
     - 0은 선형 관계가 없음을 나타냄.

    Args:
        y_true (np.ndarray): 실제 값의 배열
        y_pred (np.ndarray): 예측 값의 배열

    Returns:
        float: 'y_true'와 'y_pred' 사이의 피어슨 상관 계수
    """
    return np.corrcoef(y_true, y_pred)[0, 1]

time: 0 ns (started: 2024-06-27 13:47:32 +09:00)


In [ ]:
show_all_columns()
set_random_seed(seed_value=42)

# Load Data

In [3]:
org = pd.read_csv("../data/electric_train.csv")
test = pd.read_csv("../data/electric_test.csv")

train = org.copy()

time: 11.6 s (started: 2024-06-27 13:47:33 +09:00)


## Feature Engineering

각 데이터에 date features 추가

In [6]:
train = add_date_features(train, 'electric_train.tm', 'electric_train')
test = add_date_features(test, 'TM', 'electric_test')

time: 6.84 s (started: 2024-06-27 13:47:45 +09:00)


이상치 제거

In [7]:
outlier_base = 7

# 전력기상지수의 이상치 탐지 (예제로 Z-score 방법 사용)
z_scores = np.abs(stats.zscore(train['electric_train.elec']))
outliers = train[z_scores > outlier_base]

# 지점번호별 월 평균 전력기상지수 계산
mean_by_station_and_month = train.groupby(['electric_train.stn', 'electric_train.month'])['electric_train.elec'].mean().reset_index()
mean_by_station_and_month.rename(columns={'electric_train.elec': 'electric_train.elec_month'}, inplace=True)

# 원본 데이터와 월별 평균값을 결합 (merge)
train = train.reset_index()
train = pd.merge(train, mean_by_station_and_month, on=['electric_train.stn', 'electric_train.month'], how='left')

# 이상치를 월별 평균값으로 대체
train['electric_train.elec'] = np.where(z_scores > outlier_base, train['electric_train.elec_month'], train['electric_train.elec'])

# 필요에 따라 사용된 임시 열 제거
train.drop(columns=['electric_train.elec_month','index'], inplace=True)

time: 4.33 s (started: 2024-06-27 13:47:52 +09:00)


시간 특성에 대해 sin, cos 변환한 값 추가

In [10]:
train = add_time_features(train, 'electric_train.hh24', 'hour')
test = add_time_features(test, 'HH24', 'hour')

time: 203 ms (started: 2024-06-27 13:47:56 +09:00)


불쾌지수 추가

In [12]:
train = calculate_thi(train, 'electric_train.nph_ta', 'electric_train.nph_hm')
test = calculate_thi(test, 'nph_ta', 'nph_hm')

time: 125 ms (started: 2024-06-27 13:47:57 +09:00)


냉방지수 추가

In [14]:
train = calculate_cdd(train, 'electric_train.nph_ta')
test = calculate_cdd(test, 'nph_ta')

time: 3.44 s (started: 2024-06-27 13:48:00 +09:00)


In [16]:
train_df = train.copy()
test_df = test.copy()

time: 797 ms (started: 2024-06-27 13:48:04 +09:00)


In [17]:
train_df

,electric_train.num,electric_train.tm,electric_train.hh24,electric_train.n,electric_train.stn,electric_train.sum_qctr,electric_train.sum_load,electric_train.n_mean_load,electric_train.nph_ta,electric_train.nph_hm,electric_train.nph_ws_10m,electric_train.nph_rn_60m,electric_train.nph_ta_chi,electric_train.weekday,electric_train.week_name,electric_train.elec,electric_train.year,electric_train.month,electric_train.day,electric_train.quarter,electric_train.week,hour_sin_time,hour_cos_time,THI,CDD
0,4821,2021-01-01 01:00:00,1,11,884,6950,751.32,68.606449,2.2,62.7,1.8,0.0,-1.0,4,0,99.56,2021,1,1,1,53,2.588190e-01,0.965926,40.481506,0.0
1,4821,2021-01-01 02:00:00,2,11,884,6950,692.60,68.606449,2.3,63.1,2.1,0.0,-0.6,4,0,91.78,2021,1,1,1,53,5.000000e-01,0.866025,40.576487,0.0
2,4821,2021-01-01 03:00:00,3,11,884,6950,597.48,68.606449,2.2,62.4,2.5,0.0,-1.3,4,0,79.17,2021,1,1,1,53,7.071068e-01,0.707107,40.517872,0.0
3,4821,2021-01-01 04:00:00,4,11,884,6950,553.48,68.606449,1.7,63.5,1.7,0.0,-0.2,4,0,73.34,2021,1,1,1,53,8.660254e-01,0.500000,39.665205,0.0
4,4821,2021-01-01 05:00:00,5,11,884,6950,526.24,68.606449,1.7,63.0,1.6,0.0,-0.8,4,0,69.73,2021,1,1,1,53,9.659258e-01,0.258819,39.728290,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7593350,20947,2022-12-31 20:00:00,20,23,671,34200,6779.84,225.461986,2.7,46.3,3.1,0.0,-0.4,5,1,130.74,2022,12,31,4,52,-8.660254e-01,0.500000,43.103699,0.0
7593351,20947,2022-12-31 21:00:00,21,23,671,34200,6802.40,225.461986,2.6,46.8,3.1,0.0,-0.5,5,1,131.18,2022,12,31,4,52,-7.071068e-01,0.707107,42.918232,0.0
7593352,20947,2022-12-31 22:00:00,22,23,671,34200,6706.68,225.461986,2.4,47.4,2.1,0.0,0.2,5,1,129.33,2022,12,31,4,52,-5.000000e-01,0.866025,42.592024,0.0
7593353,20947,2022-12-31 23:00:00,23,23,671,34200,6355.88,225.461986,2.5,47.0,2.1,0.0,0.3,5,1,122.57,2022,12,31,4,52,-2.588190e-01,0.965926,42.767250,0.0


time: 16 ms (started: 2024-06-27 13:48:05 +09:00)


In [18]:
test_df

,NUM,TM,HH24,STN,nph_ta,nph_hm,nph_ws_10m,nph_rn_60m,nph_ta_chi,weekday,week_name,elect,electric_test.year,electric_test.month,electric_test.day,electric_test.quarter,electric_test.week,hour_sin_time,hour_cos_time,THI,CDD
0,4816,2023-01-01 01:00:00,1,752,3.0,68.6,2.9,0.0,-0.1,6,1.0,NaN,2023,1,1,1,52,2.588190e-01,0.965926,40.957620,0.0
1,4816,2023-01-01 02:00:00,2,752,3.1,69.4,2.7,0.0,0.3,6,1.0,NaN,2023,1,1,1,52,5.000000e-01,0.866025,41.016686,0.0
2,4816,2023-01-01 03:00:00,3,752,3.6,68.3,2.3,0.0,1.2,6,1.0,NaN,2023,1,1,1,52,7.071068e-01,0.707107,41.883312,0.0
3,4816,2023-01-01 04:00:00,4,752,4.0,69.2,3.1,0.0,1.1,6,1.0,NaN,2023,1,1,1,52,8.660254e-01,0.500000,42.384720,0.0
4,4816,2023-01-01 05:00:00,5,752,4.2,69.5,2.5,0.0,2.0,6,1.0,NaN,2023,1,1,1,52,9.659258e-01,0.258819,42.653310,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2829473,12322,2023-12-31 20:00:00,20,901,5.6,70.8,3.2,0.0,5.8,6,1.0,NaN,2023,12,31,4,52,-8.660254e-01,0.500000,44.636752,0.0
2829474,12322,2023-12-31 21:00:00,21,901,5.3,69.1,3.6,0.0,4.5,6,1.0,NaN,2023,12,31,4,52,-7.071068e-01,0.707107,44.337377,0.0
2829475,12322,2023-12-31 22:00:00,22,901,5.1,70.6,3.0,0.0,5.1,6,1.0,NaN,2023,12,31,4,52,-5.000000e-01,0.866025,43.899794,0.0
2829476,12322,2023-12-31 23:00:00,23,901,5.2,69.7,3.8,0.0,5.1,6,1.0,NaN,2023,12,31,4,52,-2.588190e-01,0.965926,44.133056,0.0


time: 16 ms (started: 2024-06-27 13:48:07 +09:00)


# Train and Valid split

- train set: 2020년, 2021년
- valid set: 2022년

In [19]:
train_df.set_index('electric_train.tm').sort_index()

,electric_train.num,electric_train.hh24,electric_train.n,electric_train.stn,electric_train.sum_qctr,electric_train.sum_load,electric_train.n_mean_load,electric_train.nph_ta,electric_train.nph_hm,electric_train.nph_ws_10m,electric_train.nph_rn_60m,electric_train.nph_ta_chi,electric_train.weekday,electric_train.week_name,electric_train.elec,electric_train.year,electric_train.month,electric_train.day,electric_train.quarter,electric_train.week,hour_sin_time,hour_cos_time,THI,CDD
electric_train.tm,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01 01:00:00,13615,1,39,140,63850,9732.96,250.490543,-8.5,74.5,0.9,0.0,-5.8,2,0,99.63,2020,1,1,1,1,2.588190e-01,0.965926,22.492325,0.0
2020-01-01 01:00:00,18235,1,20,565,14750,2401.12,116.010234,-8.8,25.6,1.9,0.0,-4.6,2,0,103.49,2020,1,1,1,1,2.588190e-01,0.965926,33.280928,0.0
2020-01-01 01:00:00,18234,1,32,565,71630,12139.16,363.252769,-8.8,25.6,1.9,0.0,-4.6,2,0,104.43,2020,1,1,1,1,2.588190e-01,0.965926,33.280928,0.0
2020-01-01 01:00:00,18233,1,28,512,45840,6723.84,225.924357,-6.3,31.3,2.5,0.0,-7.8,2,0,106.29,2020,1,1,1,1,2.588190e-01,0.965926,34.768919,0.0
2020-01-01 01:00:00,11272,1,44,941,76450,12949.24,287.250770,2.4,68.0,1.2,0.0,-5.3,2,0,102.45,2020,1,1,1,1,2.588190e-01,0.965926,40.135680,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-01 00:00:00,12468,24,40,152,50075,8018.88,187.277655,7.1,60.2,4.5,0.0,-2.3,6,1,107.05,2023,1,1,1,52,-2.449294e-16,1.000000,47.673858,0.0
2023-01-01 00:00:00,16453,24,57,617,85000,13949.76,228.064905,-5.5,81.1,0.2,0.0,-2.9,6,1,107.31,2023,1,1,1,52,-2.449294e-16,1.000000,25.831805,0.0
2023-01-01 00:00:00,11418,24,36,253,63735,11472.28,296.369472,5.6,59.5,1.3,0.0,-1.6,6,1,107.53,2023,1,1,1,52,-2.449294e-16,1.000000,45.626180,0.0


time: 1.52 s (started: 2024-06-27 13:48:16 +09:00)


train_df에서 각 격자넘버 별로 관측 기간 확인

In [20]:
tmp = train_df.groupby(by='electric_train.stn')['electric_train.tm'].agg(func=['max', 'min'])
tmp['기간'] = tmp['min'].dt.strftime('%Y-%m-%d') + ' ~ ' + tmp['max'].dt.strftime('%Y-%m-%d')

tmp

,max,min,기간
electric_train.stn,,,
98,2023-01-01,2020-01-01 01:00:00,2020-01-01 ~ 2023-01-01
99,2021-01-01,2020-01-01 01:00:00,2020-01-01 ~ 2021-01-01
101,2023-01-01,2020-01-01 01:00:00,2020-01-01 ~ 2023-01-01
104,2023-01-01,2020-01-01 01:00:00,2020-01-01 ~ 2023-01-01
106,2022-01-01,2020-01-01 01:00:00,2020-01-01 ~ 2022-01-01
...,...,...,...
942,2023-01-01,2020-01-01 01:00:00,2020-01-01 ~ 2023-01-01
943,2023-01-01,2020-01-01 01:00:00,2020-01-01 ~ 2023-01-01
950,2023-01-01,2020-01-01 01:00:00,2020-01-01 ~ 2023-01-01


time: 140 ms (started: 2024-06-27 13:48:18 +09:00)


대부분 2020년 1월 1일부터 2023년 1월 1일까지 관측한 것을 확인

In [21]:
tmp['기간'].value_counts()

2020-01-01 ~ 2023-01-01    158
2020-01-01 ~ 2022-01-01     12
2020-01-01 ~ 2021-01-01      6
2021-01-01 ~ 2022-01-01      5
2021-01-01 ~ 2023-01-01      3
2022-01-01 ~ 2023-01-01      1
Name: 기간, dtype: int64

time: 0 ns (started: 2024-06-27 13:48:20 +09:00)


따라서, 관측기간이 2020년 1월 1일부터 2023년 1월 1일까지인 데이터만을 사용

In [24]:
mask = tmp[tmp['기간'] == '2020-01-01 ~ 2023-01-01'].index

mask

Int64Index([ 98, 101, 104, 108, 112, 114, 119, 127, 129, 131,
            ...
            937, 938, 939, 940, 941, 942, 943, 950, 974, 991],
           dtype='int64', name='electric_train.stn', length=158)

time: 16 ms (started: 2024-06-27 13:48:25 +09:00)


In [25]:
data = train_df[train_df['electric_train.stn'].isin(mask)]
data = data.reset_index(drop=True)

time: 1.5 s (started: 2024-06-27 13:48:27 +09:00)


In [26]:
data

,electric_train.num,electric_train.tm,electric_train.hh24,electric_train.n,electric_train.stn,electric_train.sum_qctr,electric_train.sum_load,electric_train.n_mean_load,electric_train.nph_ta,electric_train.nph_hm,electric_train.nph_ws_10m,electric_train.nph_rn_60m,electric_train.nph_ta_chi,electric_train.weekday,electric_train.week_name,electric_train.elec,electric_train.year,electric_train.month,electric_train.day,electric_train.quarter,electric_train.week,hour_sin_time,hour_cos_time,THI,CDD
0,5565,2020-01-01 01:00:00,1,58,184,42250,4468.72,79.776051,2.3,65.8,1.7,0.0,-0.3,2,0,96.58,2020,1,1,1,1,2.588190e-01,0.965926,40.251866,0.0
1,5565,2020-01-01 02:00:00,2,58,184,42250,4101.00,79.776051,2.5,69.7,1.8,0.0,-0.2,2,0,88.63,2020,1,1,1,1,5.000000e-01,0.866025,40.082975,0.0
2,5565,2020-01-01 03:00:00,3,58,184,42250,3724.60,79.776051,2.8,69.5,2.3,0.0,-0.2,2,0,80.50,2020,1,1,1,1,7.071068e-01,0.707107,40.556040,0.0
3,5565,2020-01-01 04:00:00,4,58,184,42250,3484.40,79.776051,2.8,66.0,1.6,0.0,0.5,2,0,75.31,2020,1,1,1,1,8.660254e-01,0.500000,40.959520,0.0
4,5565,2020-01-01 05:00:00,5,58,184,42250,3409.12,79.776051,3.0,66.1,1.6,0.0,0.5,2,0,73.68,2020,1,1,1,1,9.659258e-01,0.258819,41.240870,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7198670,20947,2022-12-31 20:00:00,20,23,671,34200,6779.84,225.461986,2.7,46.3,3.1,0.0,-0.4,5,1,130.74,2022,12,31,4,52,-8.660254e-01,0.500000,43.103699,0.0
7198671,20947,2022-12-31 21:00:00,21,23,671,34200,6802.40,225.461986,2.6,46.8,3.1,0.0,-0.5,5,1,131.18,2022,12,31,4,52,-7.071068e-01,0.707107,42.918232,0.0
7198672,20947,2022-12-31 22:00:00,22,23,671,34200,6706.68,225.461986,2.4,47.4,2.1,0.0,0.2,5,1,129.33,2022,12,31,4,52,-5.000000e-01,0.866025,42.592024,0.0
7198673,20947,2022-12-31 23:00:00,23,23,671,34200,6355.88,225.461986,2.5,47.0,2.1,0.0,0.3,5,1,122.57,2022,12,31,4,52,-2.588190e-01,0.965926,42.767250,0.0


time: 16 ms (started: 2024-06-27 13:48:29 +09:00)


data의 column name 정리

In [28]:
data = clean_column_name_train(data)

time: 297 ms (started: 2024-06-27 13:48:34 +09:00)


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7198675 entries, 0 to 7198674
Data columns (total 25 columns):
 #   Column         Dtype         
---  ------         -----         
 0   NUM            int64         
 1   TM             datetime64[ns]
 2   HH24           int64         
 3   n              int64         
 4   STN            int64         
 5   sum_qctr       int64         
 6   sum_load       float64       
 7   n_mean_load    float64       
 8   nph_ta         float64       
 9   nph_hm         float64       
 10  nph_ws_10m     float64       
 11  nph_rn_60m     float64       
 12  nph_ta_chi     float64       
 13  weekday        int64         
 14  week_name      int64         
 15  elec           float64       
 16  year           int64         
 17  month          int64         
 18  day            int64         
 19  quarter        int64         
 20  week           int32         
 21  hour_sin_time  float64       
 22  hour_cos_time  float64       
 23  THI    

mask로 filtering한 데이터 중 2020년과 2021년 데이터를 train_data로, 2022년 관측 데이터를 valid_data로 선택

In [31]:
train_data, valid_data = data[data['TM'].dt.year < 2022], data[data['TM'].dt.year >= 2022]

time: 1.11 s (started: 2024-06-27 13:48:39 +09:00)


train_data 중 학습에 사용하지 않을 feature 제거

In [32]:
train_remove_columns = ['NUM', 'elec', 'TM', 'n', 'sum_qctr', 'sum_load', 'n_mean_load']

train_features = [column for column in train_data.columns if column not in train_remove_columns]

time: 0 ns (started: 2024-06-27 13:48:41 +09:00)


In [35]:
train_features

['HH24',
 'STN',
 'nph_ta',
 'nph_hm',
 'nph_ws_10m',
 'nph_rn_60m',
 'nph_ta_chi',
 'weekday',
 'week_name',
 'year',
 'month',
 'day',
 'quarter',
 'week',
 'hour_sin_time',
 'hour_cos_time',
 'THI',
 'CDD']

time: 0 ns (started: 2024-06-27 13:48:45 +09:00)


test_df 중 종속변수인 'elect'와 사용하지 않을 feature인 'TM' 제거

In [33]:
test_df.columns

Index(['NUM', 'TM', 'HH24', 'STN', 'nph_ta', 'nph_hm', 'nph_ws_10m',
       'nph_rn_60m', 'nph_ta_chi', 'weekday', 'week_name', 'elect',
       'electric_test.year', 'electric_test.month', 'electric_test.day',
       'electric_test.quarter', 'electric_test.week', 'hour_sin_time',
       'hour_cos_time', 'THI', 'CDD'],
      dtype='object')

time: 0 ns (started: 2024-06-27 13:48:42 +09:00)


In [34]:
test_remove_columns = ['elect', 'TM']

test_features = [column for column in test_df.columns if column not in test_remove_columns]

time: 0 ns (started: 2024-06-27 13:48:44 +09:00)


In [36]:
test_features

['NUM',
 'HH24',
 'STN',
 'nph_ta',
 'nph_hm',
 'nph_ws_10m',
 'nph_rn_60m',
 'nph_ta_chi',
 'weekday',
 'week_name',
 'electric_test.year',
 'electric_test.month',
 'electric_test.day',
 'electric_test.quarter',
 'electric_test.week',
 'hour_sin_time',
 'hour_cos_time',
 'THI',
 'CDD']

time: 0 ns (started: 2024-06-27 13:48:49 +09:00)


선택한 feature로 X_train, X_valid, y_train, y_valid를 만들고, 범주형 특성도 골라냄

In [37]:
X_train = train_data[train_features]
y_train = train_data['elec']
X_valid = valid_data[train_features]
y_valid = valid_data['elec']


categorical_features = ['weekday', 'week_name', 'quarter', 'STN']
numeric_features = [feature for feature in train_features if feature not in categorical_features]

time: 422 ms (started: 2024-06-27 13:48:52 +09:00)


마찬가지로, 최종 예측 테스트세트도 만듦

In [38]:
X_test = test_df[test_features]

time: 187 ms (started: 2024-06-27 13:48:54 +09:00)


앞서 지정한 범주형 특성은 OneHotEncoding을, 그외 수치형 특성은 MinMaxScaling 변환

In [39]:
numeric_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 칼럼 변환기 설정
preprocessor = ColumnTransformer(
    transformers=[
        ('NUM', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# 변환기 적용
X_train = preprocessor.fit_transform(X_train)
X_valid = preprocessor.transform(X_valid)

print(X_train.shape)
print(X_valid.shape)

(4894519, 185)
(2304156, 185)
time: 8.45 s (started: 2024-06-27 13:48:57 +09:00)


X_test의 column name이 앞서 지정한 이름과 달라 통일시켜줌

In [40]:
X_test.columns

Index(['NUM', 'HH24', 'STN', 'nph_ta', 'nph_hm', 'nph_ws_10m', 'nph_rn_60m',
       'nph_ta_chi', 'weekday', 'week_name', 'electric_test.year',
       'electric_test.month', 'electric_test.day', 'electric_test.quarter',
       'electric_test.week', 'hour_sin_time', 'hour_cos_time', 'THI', 'CDD'],
      dtype='object')

time: 0 ns (started: 2024-06-27 13:49:06 +09:00)


In [41]:
for column in X_test.columns:
    if column.startswith('electric_test.'):
        X_test = X_test.rename(columns={column:column.split('.')[1]})

time: 406 ms (started: 2024-06-27 13:49:06 +09:00)


In [42]:
X_test.columns

Index(['NUM', 'HH24', 'STN', 'nph_ta', 'nph_hm', 'nph_ws_10m', 'nph_rn_60m',
       'nph_ta_chi', 'weekday', 'week_name', 'year', 'month', 'day', 'quarter',
       'week', 'hour_sin_time', 'hour_cos_time', 'THI', 'CDD'],
      dtype='object')

time: 0 ns (started: 2024-06-27 13:49:08 +09:00)


X_test도 앞서 만든 변환기로 변환시켜줌

In [43]:
X_test = preprocessor.transform(X_test)

print(X_test.shape)

(2829478, 185)
time: 2.19 s (started: 2024-06-27 13:49:09 +09:00)


# Hyperparameter 튜닝

해당 시계열 데이터에 잘 어울리는 모델로 다음과 같은 3가지를 선택
- CatBoostRegressor
- LGBMRegressor
- XGBoostRegressor

## CatBoostRegressor

In [39]:
def objective_catboost(trial):
    cbrm_param = {
        'iterations': trial.suggest_int("iterations", 4000, 25000),
        'od_wait': trial.suggest_int('od_wait', 500, 2300),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 100, log=True),
        'subsample': trial.suggest_float('subsample', 0, 1),
        'random_strength': trial.suggest_float('random_strength', 10, 50),
        'depth': trial.suggest_int('depth', 1, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 30),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations', 1, 15),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.01, 100, log=True),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.4, 1.0),
    }

    model_cbrm = CatBoostRegressor(**cbrm_param, verbose=0)
    model_cbrm.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=25, verbose=100)
    predictions = model_cbrm.predict(X_valid)
    rscore = np.corrcoef(y_valid.squeeze(), predictions.squeeze())[0, 1]
    return rscore

study_catboost = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study_catboost.optimize(objective_catboost, n_trials=50)

[I 2024-06-26 22:49:19,264] A new study created in memory with name: no-name-8dbde11b-8c36-471c-ada7-6f853c5665cb


0:	learn: 22.1198543	test: 22.8981295	best: 22.8981295 (0)	total: 134ms	remaining: 26m 26s
100:	learn: 9.9392260	test: 10.2409679	best: 10.2409679 (100)	total: 11.7s	remaining: 22m 45s
200:	learn: 9.4040252	test: 9.6599507	best: 9.6599507 (200)	total: 23.3s	remaining: 22m 31s
300:	learn: 9.2255396	test: 9.4594528	best: 9.4586674 (296)	total: 35.1s	remaining: 22m 27s
400:	learn: 9.1385554	test: 9.3584713	best: 9.3577231 (399)	total: 46.4s	remaining: 22m 6s
500:	learn: 9.0843719	test: 9.3080886	best: 9.3060003 (499)	total: 57.6s	remaining: 21m 47s
600:	learn: 9.0455040	test: 9.2690153	best: 9.2690153 (600)	total: 1m 8s	remaining: 21m 28s
700:	learn: 9.0150100	test: 9.2483838	best: 9.2427153 (694)	total: 1m 19s	remaining: 21m 12s
Stopped by overfitting detector  (25 iterations wait)

bestTest = 9.242715276
bestIteration = 694

Shrink model to first 695 iterations.


[I 2024-06-26 22:50:46,586] Trial 0 finished with value: 0.9332660549403757 and parameters: {'iterations': 11865, 'od_wait': 2212, 'learning_rate': 0.29106359131330695, 'reg_lambda': 0.1550991398759431, 'subsample': 0.15601864044243652, 'random_strength': 16.239780813448107, 'depth': 1, 'min_data_in_leaf': 26, 'leaf_estimation_iterations': 10, 'bagging_temperature': 6.79657809075816, 'colsample_bylevel': 0.41235069657748147}. Best is trial 0 with value: 0.9332660549403757.


0:	learn: 24.3579018	test: 25.0994847	best: 25.0994847 (0)	total: 247ms	remaining: 1h 40m 29s
100:	learn: 7.8974369	test: 8.1925407	best: 8.1925407 (100)	total: 28s	remaining: 1h 52m 17s
200:	learn: 6.5047026	test: 7.0992337	best: 7.0992337 (200)	total: 59.8s	remaining: 1h 59m 52s
300:	learn: 6.0802822	test: 6.8892220	best: 6.8892220 (300)	total: 1m 24s	remaining: 1h 52m 40s
400:	learn: 5.7956305	test: 6.8403765	best: 6.8339083 (399)	total: 1m 49s	remaining: 1h 49m 10s
Stopped by overfitting detector  (25 iterations wait)

bestTest = 6.825357003
bestIteration = 430

Shrink model to first 431 iterations.


[I 2024-06-26 22:52:55,671] Trial 1 finished with value: 0.964398770257953 and parameters: {'iterations': 24369, 'od_wait': 1999, 'learning_rate': 0.0265875439832727, 'reg_lambda': 0.00018740223688836313, 'subsample': 0.18340450985343382, 'random_strength': 22.16968971838151, 'depth': 8, 'min_data_in_leaf': 13, 'leaf_estimation_iterations': 5, 'bagging_temperature': 2.801635158716261, 'colsample_bylevel': 0.4836963163912251}. Best is trial 1 with value: 0.964398770257953.


...

0:	learn: 24.3276726	test: 25.0492800	best: 25.0492800 (0)	total: 306ms	remaining: 33m 8s
100:	learn: 8.7622922	test: 9.0488476	best: 9.0488476 (100)	total: 30.3s	remaining: 31m 54s
200:	learn: 7.4086215	test: 7.6820136	best: 7.6820136 (200)	total: 1m	remaining: 31m 23s
300:	learn: 6.8395839	test: 7.2014032	best: 7.2014032 (300)	total: 1m 29s	remaining: 30m 37s
400:	learn: 6.5461768	test: 7.0536781	best: 7.0536781 (400)	total: 1m 58s	remaining: 29m 56s
500:	learn: 6.2876233	test: 6.9479911	best: 6.9477497 (498)	total: 2m 27s	remaining: 29m 18s
600:	learn: 6.0964137	test: 6.8884385	best: 6.8884336 (599)	total: 2m 56s	remaining: 28m 50s
700:	learn: 5.9281609	test: 6.8450930	best: 6.8450930 (700)	total: 3m 26s	remaining: 28m 25s
Stopped by overfitting detector  (25 iterations wait)

bestTest = 6.843315323
bestIteration = 703

Shrink model to first 704 iterations.


[I 2024-06-27 01:09:15,617] Trial 48 finished with value: 0.9642453005945273 and parameters: {'iterations': 6491, 'od_wait': 2156, 'learning_rate': 0.030501040555902418, 'reg_lambda': 2.421830916775544, 'subsample': 0.5990230437940431, 'random_strength': 31.339338342419776, 'depth': 5, 'min_data_in_leaf': 15, 'leaf_estimation_iterations': 14, 'bagging_temperature': 0.017497040793714173, 'colsample_bylevel': 0.5830046823943319}. Best is trial 43 with value: 0.9656936754794401.


0:	learn: 24.3016799	test: 25.0642735	best: 25.0642735 (0)	total: 133ms	remaining: 15m 45s
100:	learn: 12.3966946	test: 12.8773943	best: 12.8773943 (100)	total: 13s	remaining: 15m
200:	learn: 11.1601364	test: 11.5774399	best: 11.5774399 (200)	total: 25.6s	remaining: 14m 41s
300:	learn: 10.5351028	test: 10.9228289	best: 10.9228289 (300)	total: 38.4s	remaining: 14m 28s
400:	learn: 10.1513050	test: 10.4943797	best: 10.4943797 (400)	total: 51.1s	remaining: 14m 14s
500:	learn: 9.9005655	test: 10.2153759	best: 10.2153759 (500)	total: 1m 3s	remaining: 13m 57s
600:	learn: 9.7265977	test: 10.0242593	best: 10.0242593 (600)	total: 1m 15s	remaining: 13m 42s
700:	learn: 9.5978496	test: 9.8866156	best: 9.8859314 (699)	total: 1m 28s	remaining: 13m 27s
800:	learn: 9.4967946	test: 9.7776380	best: 9.7776380 (800)	total: 1m 40s	remaining: 13m 13s
900:	learn: 9.4170619	test: 9.6925798	best: 9.6925798 (900)	total: 1m 53s	remaining: 13m
1000:	learn: 9.3533536	test: 9.6241970	best: 9.6241970 (1000)	total: 2m

[I 2024-06-27 01:16:27,572] Trial 49 finished with value: 0.933903897513787 and parameters: {'iterations': 7112, 'od_wait': 1951, 'learning_rate': 0.05968788493640818, 'reg_lambda': 0.19397665467235214, 'subsample': 0.4840591597883353, 'random_strength': 41.24243483693198, 'depth': 1, 'min_data_in_leaf': 17, 'leaf_estimation_iterations': 13, 'bagging_temperature': 0.26861946935998265, 'colsample_bylevel': 0.5422126362824453}. Best is trial 43 with value: 0.9656936754794401.


time: 2h 27min 8s (started: 2024-06-26 22:49:19 +09:00)


In [53]:
print(f"Best trial: {study_catboost.best_trial.value}")
print(f"Best parameters: {study_catboost.best_trial.params}")

Best trial: 0.9656936754794401
Best parameters: {'iterations': 6644, 'od_wait': 1695, 'learning_rate': 0.06237746134009798, 'reg_lambda': 2.6397327403374717, 'subsample': 0.4915506441230588, 'random_strength': 48.24674432037646, 'depth': 4, 'min_data_in_leaf': 3, 'leaf_estimation_iterations': 15, 'bagging_temperature': 0.07131640531121744, 'colsample_bylevel': 0.4261590739334049}
time: 0 ns (started: 2024-06-27 01:56:22 +09:00)


## XGBRegressor

In [54]:
def objective_xgboost(trial):
    xgb_param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 5),
    }

    model_xgb = XGBRegressor(**xgb_param, objective='reg:squarederror')
    model_xgb.fit(X_train, y_train, 
                  eval_set=[(X_valid, y_valid)], 
                  early_stopping_rounds=25, 
                  verbose=False)
    
    predictions = model_xgb.predict(X_valid)
    rscore = np.corrcoef(y_valid.squeeze(), predictions.squeeze())[0, 1]
    return rscore

study_xgboost = optuna.create_study(direction='minimize', sampler=TPESampler(seed=42))
study_xgboost.optimize(objective_xgboost, n_trials=50)

print(f"Best trial: {study_xgboost.best_trial.value}")
print(f"Best parameters: {study_xgboost.best_trial.params}")

[I 2024-06-27 01:56:24,274] A new study created in memory with name: no-name-7bdc766c-faaa-4e32-8cba-f9883b1c040a
c:\ProgramData\Anaconda3\envs\wpy\lib\site-packages\xgboost\sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,
[I 2024-06-27 01:56:57,721] Trial 0 finished with value: 0.9601316340876372 and parameters: {'n_estimators': 437, 'learning_rate': 0.28570714885887566, 'max_depth': 8, 'min_child_weight': 6, 'subsample': 0.5780093202212182, 'colsample_bytree': 0.5779972601681014, 'gamma': 0.2904180608409973, 'reg_alpha': 4.330880728874676, 'reg_lambda': 3.005575058716044}. Best is trial 0 with value: 0.9601316340876372.
[I 2024-06-27 02:05:26,030] Trial 1 finished with value: 0.9626324292371459 and parameters: {'n_estimators': 737, 'learning_rate': 0.01596950334578271, 'max_depth': 10, 'min_child_weight': 9, 'subsample': 0.606

Best trial: 0.9570683878417976
Best parameters: {'n_estimators': 934, 'learning_rate': 0.28175665598309774, 'max_depth': 9, 'min_child_weight': 1, 'subsample': 0.9292853154718507, 'colsample_bytree': 0.9424638756724492, 'gamma': 2.0270422174538107, 'reg_alpha': 0.5941655845608484, 'reg_lambda': 2.145028502301874}
time: 1h 14min 1s (started: 2024-06-27 01:56:24 +09:00)


## LGBMRegressor

In [55]:
def objective_lightgbm(trial):
    lgb_param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'max_depth': trial.suggest_int('max_depth', -1, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 50),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0, 5),
        'lambda_l2': trial.suggest_float('lambda_l2', 0, 5),
    }

    model_lgb = LGBMRegressor(**lgb_param)
    model_lgb.fit(X_train, y_train, 
                  eval_set=[(X_valid, y_valid)])
    
    predictions = model_lgb.predict(X_valid)
    rscore = np.corrcoef(y_valid.squeeze(), predictions.squeeze())[0, 1]
    return rscore

study_lightgbm = optuna.create_study(direction='minimize', sampler=TPESampler(seed=42))
study_lightgbm.optimize(objective_lightgbm, n_trials=50)

print(f"Best trial: {study_lightgbm.best_trial.value}")
print(f"Best parameters: {study_lightgbm.best_trial.params}")

[I 2024-06-27 03:10:25,668] A new study created in memory with name: no-name-0bc88d1a-d383-4e1c-98b1-a3015e175d7f


[LightGBM] [Warning] min_data_in_leaf is set=8, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=8
[LightGBM] [Warning] feature_fraction is set=0.49359671220172163, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.49359671220172163
[LightGBM] [Warning] lambda_l1 is set=3.005575058716044, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.005575058716044
[LightGBM] [Warning] lambda_l2 is set=3.540362888980227, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.540362888980227
[LightGBM] [Warning] bagging_fraction is set=0.4348501673009197, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4348501673009197
[LightGBM] [Warning] bagging_freq is set=9, subsample_freq=0 will be ignored. Current value: bagging_freq=9
[LightGBM] [Warning] min_data_in_leaf is set=8, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=8
[LightGBM] [Warning] feature_fraction is set=0.49359671220172163, colsample_bytree=1.0

[I 2024-06-27 03:11:21,153] Trial 0 finished with value: 0.9586937923877085 and parameters: {'n_estimators': 437, 'learning_rate': 0.28570714885887566, 'num_leaves': 188, 'max_depth': 9, 'min_data_in_leaf': 8, 'feature_fraction': 0.49359671220172163, 'bagging_fraction': 0.4348501673009197, 'bagging_freq': 9, 'lambda_l1': 3.005575058716044, 'lambda_l2': 3.540362888980227}. Best is trial 0 with value: 0.9586937923877085.


[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] feature_fraction is set=0.5100427059120604, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5100427059120604
[LightGBM] [Warning] lambda_l1 is set=2.1597250932105787, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.1597250932105787
[LightGBM] [Warning] lambda_l2 is set=1.4561457009902097, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.4561457009902097
[LightGBM] [Warning] bagging_fraction is set=0.5825453457757226, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5825453457757226
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] feature_fraction is set=0.5100427059120604, colsample_bytre

[I 2024-06-27 03:11:29,563] Trial 1 finished with value: 0.9555601755416646 and parameters: {'n_estimators': 118, 'learning_rate': 0.29127385712697834, 'num_leaves': 214, 'max_depth': 2, 'min_data_in_leaf': 10, 'feature_fraction': 0.5100427059120604, 'bagging_fraction': 0.5825453457757226, 'bagging_freq': 6, 'lambda_l1': 2.1597250932105787, 'lambda_l2': 1.4561457009902097}. Best is trial 1 with value: 0.9555601755416646.


...

[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] feature_fraction is set=0.8240406890951261, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8240406890951261
[LightGBM] [Warning] lambda_l1 is set=0.8892036978267366, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8892036978267366
[LightGBM] [Warning] lambda_l2 is set=4.228305239103268, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.228305239103268
[LightGBM] [Warning] bagging_fraction is set=0.8743814446734592, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8743814446734592
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] feature_fraction is set=0.8240406890951261, colsample_bytree=

[I 2024-06-27 04:20:18,702] Trial 48 finished with value: 0.9338036781556204 and parameters: {'n_estimators': 744, 'learning_rate': 0.2545244786177426, 'num_leaves': 246, 'max_depth': 1, 'min_data_in_leaf': 10, 'feature_fraction': 0.8240406890951261, 'bagging_fraction': 0.8743814446734592, 'bagging_freq': 5, 'lambda_l1': 0.8892036978267366, 'lambda_l2': 4.228305239103268}. Best is trial 48 with value: 0.9338036781556204.


[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11
[LightGBM] [Warning] feature_fraction is set=0.817072661897291, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.817072661897291
[LightGBM] [Warning] lambda_l1 is set=0.8367563018412693, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8367563018412693
[LightGBM] [Warning] lambda_l2 is set=4.998220284984137, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.998220284984137
[LightGBM] [Warning] bagging_fraction is set=0.8739444832072618, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8739444832072618
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11
[LightGBM] [Warning] feature_fraction is set=0.817072661897291, colsample_bytree=1.0

[I 2024-06-27 04:20:50,175] Trial 49 finished with value: 0.9335688013568963 and parameters: {'n_estimators': 672, 'learning_rate': 0.2540899515936272, 'num_leaves': 112, 'max_depth': 1, 'min_data_in_leaf': 11, 'feature_fraction': 0.817072661897291, 'bagging_fraction': 0.8739444832072618, 'bagging_freq': 6, 'lambda_l1': 0.8367563018412693, 'lambda_l2': 4.998220284984137}. Best is trial 49 with value: 0.9335688013568963.


Best trial: 0.9335688013568963
Best parameters: {'n_estimators': 672, 'learning_rate': 0.2540899515936272, 'num_leaves': 112, 'max_depth': 1, 'min_data_in_leaf': 11, 'feature_fraction': 0.817072661897291, 'bagging_fraction': 0.8739444832072618, 'bagging_freq': 6, 'lambda_l1': 0.8367563018412693, 'lambda_l2': 4.998220284984137}
time: 1h 10min 24s (started: 2024-06-27 03:10:25 +09:00)


# Model Compare

In [61]:
study_catboost.best_params

{'iterations': 6644,
 'od_wait': 1695,
 'learning_rate': 0.06237746134009798,
 'reg_lambda': 2.6397327403374717,
 'subsample': 0.4915506441230588,
 'random_strength': 48.24674432037646,
 'depth': 4,
 'min_data_in_leaf': 3,
 'leaf_estimation_iterations': 15,
 'bagging_temperature': 0.07131640531121744,
 'colsample_bylevel': 0.4261590739334049}

time: 0 ns (started: 2024-06-27 07:21:05 +09:00)


In [45]:
study_catboost_best_params = {'iterations': 6644,
                              'od_wait': 1695,
                              'learning_rate': 0.06237746134009798,
                              'reg_lambda': 2.6397327403374717,
                              'subsample': 0.4915506441230588,
                              'random_strength': 48.24674432037646,
                              'depth': 4,
                              'min_data_in_leaf': 3,
                              'leaf_estimation_iterations': 15,
                              'bagging_temperature': 0.07131640531121744,
                              'colsample_bylevel': 0.4261590739334049}

time: 0 ns (started: 2024-06-27 13:49:54 +09:00)


In [65]:
study_lightgbm.best_params

{'n_estimators': 672,
 'learning_rate': 0.2540899515936272,
 'num_leaves': 112,
 'max_depth': 1,
 'min_data_in_leaf': 11,
 'feature_fraction': 0.817072661897291,
 'bagging_fraction': 0.8739444832072618,
 'bagging_freq': 6,
 'lambda_l1': 0.8367563018412693,
 'lambda_l2': 4.998220284984137}

time: 0 ns (started: 2024-06-27 07:24:09 +09:00)


In [46]:
study_lightgbm_best_params = {'n_estimators': 672,
                              'learning_rate': 0.2540899515936272,
                              'num_leaves': 112,
                              'max_depth': 1,
                              'min_data_in_leaf': 11,
                              'feature_fraction': 0.817072661897291,
                              'bagging_fraction': 0.8739444832072618,
                              'bagging_freq': 6,
                              'lambda_l1': 0.8367563018412693,
                              'lambda_l2': 4.998220284984137}

time: 0 ns (started: 2024-06-27 13:50:12 +09:00)


In [67]:
study_xgboost.best_params

{'n_estimators': 934,
 'learning_rate': 0.28175665598309774,
 'max_depth': 9,
 'min_child_weight': 1,
 'subsample': 0.9292853154718507,
 'colsample_bytree': 0.9424638756724492,
 'gamma': 2.0270422174538107,
 'reg_alpha': 0.5941655845608484,
 'reg_lambda': 2.145028502301874}

time: 0 ns (started: 2024-06-27 12:21:05 +09:00)


In [47]:
study_xgboost_best_params = {'n_estimators': 934,
                             'learning_rate': 0.28175665598309774,
                             'max_depth': 9,
                             'min_child_weight': 1,
                             'subsample': 0.9292853154718507,
                             'colsample_bytree': 0.9424638756724492,
                             'gamma': 2.0270422174538107,
                             'reg_alpha': 0.5941655845608484,
                             'reg_lambda': 2.145028502301874}

time: 0 ns (started: 2024-06-27 13:50:30 +09:00)


## Eval Models

In [77]:
def pearson_corrcoef(y_true, y_pred):
    return np.corrcoef(y_true.squeeze(), y_pred.squeeze())[0, 1]

def evaluate_model(model, X_train, X_valid, y_train, y_valid, early_stopping=True):
    if early_stopping:
        model.fit(X_train, y_train, 
                  eval_set=[(X_valid, y_valid)], 
                  early_stopping_rounds=25, 
                  verbose=False)
    else:
        model.fit(X_train, y_train, 
                  eval_set=[(X_valid, y_valid)])
    
    predictions = model.predict(X_valid)
    rscore = pearson_corrcoef(y_valid, predictions)
    return rscore

catboost_model = CatBoostRegressor(**study_catboost.best_params)
xgboost_model = XGBRegressor(**study_xgboost.best_params)
lightgbm_model = LGBMRegressor(**study_lightgbm.best_params)

catboost_rscore = evaluate_model(catboost_model, X_train, X_valid, y_train, y_valid, early_stopping=True)
xgboost_rscore = evaluate_model(xgboost_model, X_train, X_valid, y_train, y_valid, early_stopping=False)
lightgbm_rscore = evaluate_model(lightgbm_model, X_train, X_valid, y_train, y_valid, early_stopping=False)

print(f"CatBoost Pearson correlation coefficient: {catboost_rscore}")
print(f"XGBoost Pearson correlation coefficient: {xgboost_rscore}")
print(f"LightGBM Pearson correlation coefficient: {lightgbm_rscore}")

[0]	validation_0-rmse:73.86474
[1]	validation_0-rmse:53.20470
[2]	validation_0-rmse:38.45468
[3]	validation_0-rmse:28.00462
[4]	validation_0-rmse:20.74678
[5]	validation_0-rmse:15.81235
[6]	validation_0-rmse:12.50904
[7]	validation_0-rmse:10.39212
[8]	validation_0-rmse:9.14442
[9]	validation_0-rmse:8.41317
[10]	validation_0-rmse:8.06978
[11]	validation_0-rmse:7.85157
[12]	validation_0-rmse:7.73178
[13]	validation_0-rmse:7.66012
[14]	validation_0-rmse:7.63827
[15]	validation_0-rmse:7.61266
[16]	validation_0-rmse:7.59197
[17]	validation_0-rmse:7.58415
[18]	validation_0-rmse:7.56603
[19]	validation_0-rmse:7.57124
[20]	validation_0-rmse:7.56109
[21]	validation_0-rmse:7.55358
[22]	validation_0-rmse:7.54548
[23]	validation_0-rmse:7.53493
[24]	validation_0-rmse:7.51740
[25]	validation_0-rmse:7.54882
[26]	validation_0-rmse:7.53553
[27]	validation_0-rmse:7.52651
[28]	validation_0-rmse:7.51966
[29]	validation_0-rmse:7.52864
[30]	validation_0-rmse:7.52156
[31]	validation_0-rmse:7.51635
[32]	valid

# Test Result

In [49]:
X_test

<2829478x185 sparse matrix of type '<class 'numpy.float64'>'
	with 44550939 stored elements in Compressed Sparse Row format>

time: 0 ns (started: 2024-06-27 13:51:49 +09:00)


X_train과 X_valid를 concat한 후 희소행렬로 만듦

In [ ]:
X_final_train = np.concatenate((X_train.todense(), X_valid.todense()), axis=0)
y_final_train = np.concatenate((y_train, y_valid), axis=0)

X_final_train_sparse = csr_matrix(X_final_train)

최종으로 얻은 best_parameter로 각 모델을 학습하고, pkl형태로 저장

In [ ]:
# Train the final CatBoost model
final_catboost_model = CatBoostRegressor(**study_catboost_best_params)
final_catboost_model.fit(X_final_train_sparse, y_final_train, verbose=False)
joblib.dump(final_catboost_model, 'final_catboost_model.pkl')

In [ ]:
# Train the final XGBoost model
final_xgboost_dmatrix = DMatrix(X_final_train_sparse, label=y_final_train)
final_xgboost_model = XGBRegressor(**study_xgboost_best_params, objective='reg:squarederror')
final_xgboost_model.fit(X_final_train_sparse, y_final_train, verbose=False)
joblib.dump(final_xgboost_model, 'final_xgboost_model.pkl')

In [ ]:
# Train the final LightGBM model
final_lightgbm_dataset = Dataset(X_final_train_sparse, label=y_final_train)
final_lightgbm_model = LGBMRegressor(**study_lightgbm_best_params)
final_lightgbm_model.fit(X_final_train_sparse, y_final_train)
joblib.dump(final_lightgbm_model, 'final_lightgbm_model.pkl')

## Final Submission

In [64]:
res = pd.read_csv('../data/electric_test.csv')

res['elect'] = final_catboost_model.predict(X_test)

res.to_csv('240312.csv', index=False)

time: 17.1 s (started: 2024-06-27 15:21:22 +09:00)


최종 예측 간 상관관계 확인

In [71]:
cat_res = final_catboost_model.predict(X_test)
xgb_res = final_xgboost_model.predict(X_test)
lgbm_res = final_lightgbm_model.predict(X_test)

[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11
[LightGBM] [Warning] feature_fraction is set=0.817072661897291, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.817072661897291
[LightGBM] [Warning] lambda_l1 is set=0.8367563018412693, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8367563018412693
[LightGBM] [Warning] lambda_l2 is set=4.998220284984137, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.998220284984137
[LightGBM] [Warning] bagging_fraction is set=0.8739444832072618, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8739444832072618
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
time: 37 s (started: 2024-06-27 15:29:40 +09:00)


In [72]:
cat_res

array([101.19458772,  89.08676444,  79.17274224, ..., 129.30724906,
       119.67231616, 108.86112048])

time: 0 ns (started: 2024-06-27 15:30:17 +09:00)


In [73]:
xgb_res

array([ 99.679184,  85.89056 ,  77.196106, ..., 130.03871 , 121.47312 ,
       109.411736], dtype=float32)

time: 0 ns (started: 2024-06-27 15:30:17 +09:00)


In [74]:
lgbm_res

array([103.49096976,  90.8188584 ,  83.39900934, ..., 134.53283669,
       126.64563799, 110.38179456])

time: 0 ns (started: 2024-06-27 15:30:17 +09:00)


In [79]:
from itertools import combinations

results = [cat_res, xgb_res, lgbm_res]

for res1, res2 in combinations(results, 2):
    print(np.corrcoef(res1, res2)[0, 1])

0.9699433887448536
0.956070214462164
0.9381451263514439
time: 94 ms (started: 2024-06-27 15:32:36 +09:00)
